In [1]:
import numpy as np
import keras
from keras import callbacks
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
img_row = 108
img_col = 108
path = "E:/database108/"
batch_size = 64
output_classes = 24

In [9]:
from math import log10, floor
def sig(number, sig=2):
    if number == 0:
        return 0
    return round(number, sig - floor(log10(abs(number))) - 1)

In [6]:
import pandas as pd
file = pd.read_csv(path + "database_517.csv")
array = file.values
print(array.shape)

(4, 11666)


In [12]:
import random
import pandas as pd
x_train_ori = []
y_train_ori = []
x_test_ori = []
y_test_ori = []
for i in range(516):
    print(i)
    file = pd.read_csv(path + "database_{}.csv".format(i), dtype=str)
    array = file.values
    array = list(array)
    train_test = random.sample(range(100), 100)
    
    for j in train_test[:90]:
        sample = list(array[j])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                x = "".join(x.split())
                #x = sig(float(x), 3)
                lst.append(float(x))
            elif type(x) == float:
                #x = sig(x, 3)
                lst.append(x)
            else:
                print(type(x))
        x_train_ori.append(lst)
        y_train_ori.append(sample[0])
        
    for j in train_test[:90]:
        sample = list(array[j])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                x = "".join(x.split())
                #x = sig(float(x), 3)
                lst.append(x)
            elif type(x) == float:
                #x = sig(x, 3)
                lst.append(float(x))
            else:
                print(type(x))
        x_train_ori.append(lst)
        y_train_ori.append(sample[0])
    
    file = None

0
1
2
3
4
5
6
7
8
9
10


KeyboardInterrupt: 

In [4]:
print(np.shape(x_train_ori))
print(np.shape(x_test_ori))
print(np.shape(y_train_ori))
print(np.shape(y_test_ori))

(25800, 13176)
(5160, 13176)
(25800,)
(5160,)


In [5]:
def y_convert(array, flag=0):
    y = []
    for i in array: # i is the name of each sample:
        name = i.split("_")
        name = name[1:5]
        if flag == 0:
            # category of appearance of a note
            category = np.zeros(24, dtype=int)
            for j in name:
                if j != "x":
                    num = int(j)
                    category[num-60] = 1
            y.append(list(category))
    return y

In [6]:
lst_1 = []
lst_2 = []

for i in y_train_ori:
    label = "".join(i.split())
    lst_1.append(label)
for j in y_test_ori:
    label = "".join(j.split())
    lst_2.append(label)
    
y_train_ori = y_convert(lst_1)
y_test_ori = y_convert(lst_2)

In [7]:
y_train_ori = np.array(y_train_ori)
y_test_ori = np.array(y_test_ori)
x_train_ori = np.array(x_train_ori)
x_test_ori = np.array(x_test_ori)

In [10]:
x_train_ori = x_train_ori.flatten()
x_train_ori = x_train_ori.reshape(25800, img_row, img_col, 1)
x_test_ori = x_test_ori.flatten()
x_test_ori = x_test_ori.reshape(5160, img_row, img_col, 1)

In [12]:
print(x_train_ori.shape)
input_shape = (108, 122, 1)

(25800, 108, 122, 1)


In [14]:
def build_model():
    filename = "model_train.csv"
    csv_log = callbacks.CSVLogger(filename, separator=",", append=False)
    
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [csv_log, early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
    return model

In [16]:
# randomly taking 10320 data samples
classifier = build_model()
hist = classifier.fit(x_train_ori, y_train_ori, batch_size=128, nb_epoch=100, verbose=1, 
                      validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/100
25800/25800 [==============================] - 22s 861us/step - loss: 12.4227 - acc: 0.0439 - val_loss: 18.6376 - val_acc: 0.0539
Epoch 2/100
25800/25800 [==============================] - 19s 724us/step - loss: 12.2350 - acc: 0.0424 - val_loss: 31.9129 - val_acc: 1.9380e-04
Epoch 3/100
25800/25800 [==============================] - 19s 726us/step - loss: 12.1625 - acc: 0.0421 - val_loss: 42.1439 - val_acc: 0.0000e+00
Epoch 4/100
25800/25800 [==============================] - 19s 726us/step - loss: 12.1245 - acc: 0.0406 - val_loss: 44.5855 - val_acc: 0.1581
Epoch 5/100
25800/25800 [==============================] - 19s 727us/step - loss: 12.1132 - acc: 0.0389 - val_loss: 49.0933 - val_acc: 0.0000e+00
Epoch 6/100
25800/25800 [==============================] - 19s 726us/step - loss: 12.1022 - acc: 0.0441 - val_loss: 51.5481 - val_acc: 0.0000e+00
Epoch 7/100
25800/25800 [==============================] - 19s 727us/step - loss: 1

KeyboardInterrupt: 

In [22]:
def build_model_A():
    filename = "modelA_train.csv"
    csv_log = callbacks.CSVLogger(filename, separator=",", append=False)
    
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [csv_log, early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
        
    #second FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [23]:
# randomly taking 25800 data samples
classifier_A = build_model_A()
hist_A = classifier_A.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=40, verbose=1, validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/40
25800/25800 [==============================] - 23s 888us/step - loss: 12.2578 - acc: 0.0419 - val_loss: 58.3998 - val_acc: 0.0027
Epoch 2/40
25800/25800 [==============================] - 21s 816us/step - loss: 12.1596 - acc: 0.0400 - val_loss: 56.6712 - val_acc: 0.0000e+00
Epoch 3/40
25800/25800 [==============================] - 21s 817us/step - loss: 12.1405 - acc: 0.0452 - val_loss: 58.6842 - val_acc: 0.0097
Epoch 4/40
25800/25800 [==============================] - 21s 819us/step - loss: 12.1280 - acc: 0.0416 - val_loss: 58.7311 - val_acc: 0.0000e+00
Epoch 5/40
25800/25800 [==============================] - 21s 820us/step - loss: 12.1260 - acc: 0.0433 - val_loss: 58.7311 - val_acc: 0.0000e+00
Epoch 6/40
25800/25800 [==============================] - 21s 819us/step - loss: 12.1156 - acc: 0.0462 - val_loss: 58.6811 - val_acc: 0.0638
Epoch 7/40
25800/25800 [==============================] - 21s 820us/step - loss: 12.1116 - ac

In [24]:
fname = "weight-Test-CNN_A.hdf5"
classifier_A.save_weights(fname, overwrite=True)

In [17]:
def build_model_B():
    filename = "model_train_B.csv"
    csv_log = callbacks.CSVLogger(filename, separator=",", append=False)
    
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [csv_log, early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
    return model

In [18]:
# randomly taking 25800 data samples
classifier_B = build_model_B()
hist_B = classifier_B.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=40, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/40
25800/25800 [==============================] - 20s 761us/step - loss: 12.3171 - acc: 0.0397 - val_loss: 12.1256 - val_acc: 0.0097
Epoch 2/40
25800/25800 [==============================] - 19s 729us/step - loss: 12.1525 - acc: 0.0407 - val_loss: 12.2068 - val_acc: 0.0446
Epoch 3/40
25800/25800 [==============================] - 19s 730us/step - loss: 12.1167 - acc: 0.0428 - val_loss: 12.3817 - val_acc: 7.7519e-04
Epoch 4/40
25800/25800 [==============================] - 19s 730us/step - loss: 12.1087 - acc: 0.0464 - val_loss: 12.1884 - val_acc: 0.0446
Epoch 5/40
25800/25800 [==============================] - 19s 729us/step - loss: 12.1081 - acc: 0.0452 - val_loss: 12.1333 - val_acc: 0.0027
Epoch 6/40
25800/25800 [==============================] - 19s 730us/step - loss: 12.1092 - acc: 0.0431 - val_loss: 13.3281 - val_acc: 0.0446
Epoch 7/40
25800/25800 [==============================] - 19s 730us/step - loss: 12.1053 - acc: 0.042

In [21]:
fname = "weight-Test-CNN_B.hdf5"
classifier_B.save_weights(fname, overwrite=True)

In [25]:
def build_model_C():
    filename = "modelA_train.csv"
    csv_log = callbacks.CSVLogger(filename, separator=",", append=False)
    
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [csv_log, early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    #model.add(Dropout(0.2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    #model.add(Dropout(0.2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    #model.add(Dropout(0.2))
    
    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    #model.add(Dropout(0.2))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    #model.add(Dropout(0.2))
        
    #second FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    #model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [26]:
classifier_C = build_model_B()
hist_C = classifier_B.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=40, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/40
25800/25800 [==============================] - 19s 747us/step - loss: 12.0989 - acc: 0.0436 - val_loss: 13.4296 - val_acc: 0.1581
Epoch 2/40
25800/25800 [==============================] - 19s 730us/step - loss: 12.1005 - acc: 0.0436 - val_loss: 12.3314 - val_acc: 0.0446
Epoch 3/40
25800/25800 [==============================] - 19s 730us/step - loss: 12.0998 - acc: 0.0437 - val_loss: 12.3964 - val_acc: 0.1581
Epoch 4/40
25800/25800 [==============================] - 19s 731us/step - loss: 12.1014 - acc: 0.0428 - val_loss: 12.6085 - val_acc: 0.0446
Epoch 5/40
25800/25800 [==============================] - 19s 731us/step - loss: 12.0999 - acc: 0.0416 - val_loss: 12.8277 - val_acc: 0.0446
Epoch 6/40
25800/25800 [==============================] - 19s 730us/step - loss: 12.0995 - acc: 0.0453 - val_loss: 12.3218 - val_acc: 0.0446
Epoch 7/40
25800/25800 [==============================] - 19s 731us/step - loss: 12.1006 - acc: 0.0421 - 

In [27]:
fname = "weight-Test-CNN_C.hdf5"
classifier_B.save_weights(fname, overwrite=True)